# Various Problem Solving algorithms
1. Tree Search
2. Graph Search
3. Breath First Search using graph
4. Depth First Search

Reference: The AIMA book, https://github.com/aimacode/aima-python

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML



## Section 1: Common infrastructure

### The map of Romania

In [22]:

romania_map = dict(
    Arad=dict(Zerind=75, Sibiu=140, Timisoara=118),
    Bucharest=dict(Urziceni=85, Pitesti=101, Giurgiu=90, Fagaras=211),
    Craiova=dict(Drobeta=120, Rimnicu=146, Pitesti=138),
    Drobeta=dict(Mehadia=75),
    Eforie=dict(Hirsova=86),
    Fagaras=dict(Sibiu=99),
    Hirsova=dict(Urziceni=98),
    Iasi=dict(Vaslui=92, Neamt=87),
    Lugoj=dict(Timisoara=111, Mehadia=70),
    Oradea=dict(Zerind=71, Sibiu=151),
    Pitesti=dict(Rimnicu=97),
    Rimnicu=dict(Sibiu=80),
    Urziceni=dict(Vaslui=142))

romania_map_locations = dict(
    Arad=(91, 492), Bucharest=(400, 327), Craiova=(253, 288),
    Drobeta=(165, 299), Eforie=(562, 293), Fagaras=(305, 449),
    Giurgiu=(375, 270), Hirsova=(534, 350), Iasi=(473, 506),
    Lugoj=(165, 379), Mehadia=(168, 339), Neamt=(406, 537),
    Oradea=(131, 571), Pitesti=(320, 368), Rimnicu=(233, 410),
    Sibiu=(207, 457), Timisoara=(94, 410), Urziceni=(456, 350),
    Vaslui=(509, 444), Zerind=(108, 531))


### The Graph class
Which encapsulates the above map (and can plot it too)

In [23]:
class Graph(object):
    
    def __init__(self, dictionary=None, locations=None, directed=True):
        self.dictionary = dictionary or {}
        self.directed = directed
        self.locations = locations
        if not self.directed:
            self.make_undirected()
        
            
    def make_undirected(self):
        for city in list(self.dictionary.keys()):
            for (target, cost) in self.dictionary[city].items():
                self.connect_nodes(city, target, cost)
    
    def connect_nodes(self, a, b, cost):
        self.connect_nodes_helper(a, b, cost)
        if not self.directed:
            self.connect_nodes_helper(b, a, cost)
    
    def connect_nodes_helper(self, a, b, cost):
        self.dictionary.setdefault(a, {})[b] = cost
        
    def get_nodes(self):
        return list(self.dictionary.keys())
    
    def get_connections(self, a, b=None):
        connections = self.dictionary.setdefault(a, {})
        if b is None:
            return connections
        else:
            return connections[b]
        
        
        

### plot() is a function that can plot a Graph object

In [24]:
# This function is not used anymore. The Plot class below is being used.

def plot(graph, solution = None, tries=None):
    if g.locations is None:
        print("No locations given. Cannot plot.")
        return

    fig = plt.figure()
    sp = fig.add_subplot(111)

    x_max = -float('inf')
    x_min = float('inf')

    y_max = -float('inf')
    y_min = float('inf')

    x = []
    y = []
    names = []

    for city in g.locations:
        if g.locations[city][0] < x_min:
            x_min = g.locations[city][0]
        if g.locations[city][0] > x_max:
            x_max = g.locations[city][0]

        if g.locations[city][1] < y_min:
            y_min = g.locations[city][1]
        if g.locations[city][1] > y_max:
            y_max = g.locations[city][1]

        x.append(g.locations[city][0])
        y.append(g.locations[city][1])
        names.append(city)


    if x_max > y_max:
        the_max = x_max
    else:
        the_max = y_max

    border = 50

    sp.set(xlim=[x_min-border, x_max+border], ylim=[y_min-border, y_max+border],
           title='Map of the place', ylabel='Y-Axis', xlabel='X-Axis')

    sp.scatter(x, y, color='orange', marker='o', s=100)

    for i, name in enumerate(names):
        sp.annotate(name, (x[i] + 4,y[i] + 4))

    for city in g.dictionary:
        for target in g.dictionary[city]:
            x1 = g.locations[city][0]
            y1 = g.locations[city][1]
            x2 = g.locations[target][0]
            y2 = g.locations[target][1]
            lines, = sp.plot([x1, x2], [y1, y2], color='gray')
            lines.set_linestyle(':')
            distance = str(g.dictionary[city][target])
            xa = min(x1, x2)
            xb = max(x1, x2)
            ya = min(y1, y2)
            yb = max(y1, y2)
            sp.annotate(distance, (xa + (xb-xa)/2, (ya + (yb-ya)/2)), color='teal')


    if solution:
        previous = None
        for step in solution:
            if previous is not None:
                x1 = g.locations[previous.state][0]
                y1 = g.locations[previous.state][1]
                x2 = g.locations[step.state][0]
                y2 = g.locations[step.state][1]
                lines, = sp.plot([x1, x2], [y1, y2], color='red')

                #lines.set_linestyle('-')
            previous = step

    #plt.tight_layout(pad=-20)

    #ani = animation.FuncAnimation(fig, animate, fargs=[5],interval = 100)

    plt.grid()
    plt.plot()
    plt.show()

In [25]:

class Plot(object):
    def __init__(self):
        self.fig = plt.figure(figsize=(15, 10))
        self.sp = self.fig.add_subplot(111)
        
        
    def plot(self, graph, solution = None, tries=None):
        if g.locations is None:
            print("No locations given. Cannot plot.")
            return

        x_max = -float('inf')
        x_min = float('inf')

        y_max = -float('inf')
        y_min = float('inf')

        x = []
        y = []
        names = []

        for city in g.locations:
            if g.locations[city][0] < x_min:
                x_min = g.locations[city][0]
            if g.locations[city][0] > x_max:
                x_max = g.locations[city][0]

            if g.locations[city][1] < y_min:
                y_min = g.locations[city][1]
            if g.locations[city][1] > y_max:
                y_max = g.locations[city][1]

            x.append(g.locations[city][0])
            y.append(g.locations[city][1])
            names.append(city)


        if x_max > y_max:
            the_max = x_max
        else:
            the_max = y_max

        border = 50

        self.sp.set(xlim=[x_min-border, x_max+border], ylim=[y_min-border, y_max+border],
               title='Map of the place', ylabel='Y-Axis', xlabel='X-Axis')

        self.sp.scatter(x, y, color='orange', marker='o', s=100)

        for i, name in enumerate(names):
            self.sp.annotate(name, (x[i] + 4,y[i] + 4))

        for city in g.dictionary:
            for target in g.dictionary[city]:
                x1 = g.locations[city][0]
                y1 = g.locations[city][1]
                x2 = g.locations[target][0]
                y2 = g.locations[target][1]
                lines, = self.sp.plot([x1, x2], [y1, y2], color='gray')
                lines.set_linestyle(':')
                distance = str(g.dictionary[city][target])
                xa = min(x1, x2)
                xb = max(x1, x2)
                ya = min(y1, y2)
                yb = max(y1, y2)
                self.sp.annotate(distance, (xa + (xb-xa)/2, (ya + (yb-ya)/2)), color='teal')


        if solution:
            previous = None
            for step in solution:
                if previous is not None:
                    x1 = g.locations[previous.state][0]
                    y1 = g.locations[previous.state][1]
                    x2 = g.locations[step.state][0]
                    y2 = g.locations[step.state][1]
                    lines, = self.sp.plot([x1, x2], [y1, y2], color='red')
                previous = step

        #plt.tight_layout(pad=-20)
        #plt.grid()
        #plt.plot()
        #plt.show()


Test it out

In [26]:

g = Graph(romania_map, romania_map_locations, False)
print(g.get_connections('Arad', 'Timisoara'))
print(g.get_connections('Urziceni'))


#p = P()
#p.plot(g)

#def animate(i):
#    s = p.sp.scatter([200+i, 300+i], [400, 500])
#    return s

#anim = animation.FuncAnimation(p.fig, animate, frames=10, interval=1000, blit=False)
#HTML(anim.to_html5_video())
    


118
{'Vaslui': 142, 'Hirsova': 98, 'Bucharest': 85}


### The GraphProblem class
Which encapsulates the problem (initial state, goal state, actions, results, path cost)

In [27]:
class GraphProblem(object):
    """The Problem Definition"""
    def __init__(self, graph, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.graph = graph
      
    
    def actions(self, state):
        return list(self.graph.get_connections(state))
    
    
    def results(self, state, action):
        if action in self.graph.get_connections(state):
            return action
        else:
            return None
        
        
    def goal_test(self, state):
        return self.goal_state == state
    
    
    def path_cost(self, c, state1, state2, action):
        return c + self.graph.get_connections(state1, state2)
        

Test it out

In [28]:
g = Graph(romania_map, romania_map_locations, False)
gp = GraphProblem(g, 'Arad', 'Bucharest')
print (gp.actions('Urziceni'))
print (gp.results('Sibiu','Arad'))
print (gp.path_cost(10, 'Arad', 'Timisoara', 'Timisoara'))

['Vaslui', 'Hirsova', 'Bucharest']
Arad
128


### The Node class
Which represents the frontiers and explored nodes. Maintains the path travelled to reach it. Path cost. Parent.

In [29]:
class Node(object):
    
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        
        if self.parent:
            self.depth = self.parent.depth + 1
            
        
    def __repr__(self):
        return "{0}".format(self.state)
    
    
    def __lt__(self, node):
        return self.state < node.state
    
    
    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state
    
    
    def __hash__(self):
        return hash(self.state)
        
    
    def expand(self, problem):
        return [self.child_node(problem, action)
            for action in problem.actions(self.state)]
    
    
    def child_node(self, problem, action):
        next_state = problem.results(self.state, action)
        return Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, next_state, action))
    
    
    def path(self):
        node = self
        path_back = []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    
    def solution(self):
        return [node.action for node in self.path()[1:]]

## Section 2: Tree Search

In [30]:
import queue

def tree_search(problem):
    
    # Initialise Queue (FIFO)
    frontier = queue.Queue()
    frontier.put(Node(problem.initial_state))
    iterations = []
    
    # Algorithm starts
    while not frontier.empty():
        path = frontier.get()
        iterations.append(path.path())
        
        if problem.goal_test(path.state):
            return True, iterations, path
        
        paths = path.expand(problem)
        for p in paths:
            frontier.put(p)
        
    return False, None, None

In [32]:

gp = GraphProblem(g, 'Arad', 'Bucharest')
result, iterations, path = tree_search(gp)
print(result)
print(path.path())
p = Plot()
p.plot(g)

for i in iterations:
    print(i)

def animate(i):
    p.plot(g, iterations[i])
    sites = iterations[i]
    for site in sites:
        x = romania_map_locations[site.state][0]
        y = romania_map_locations[site.state][1]
        s = p.sp.scatter(x, y, color='red', marker='o', s=100)
    return s

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=3000, blit=False)
HTML(anim.to_html5_video())
    


True
[Arad, Sibiu, Fagaras, Bucharest]
[Arad]
[Arad, Timisoara]
[Arad, Zerind]
[Arad, Sibiu]
[Arad, Timisoara, Lugoj]
[Arad, Timisoara, Arad]
[Arad, Zerind, Oradea]
[Arad, Zerind, Arad]
[Arad, Sibiu, Rimnicu]
[Arad, Sibiu, Fagaras]
[Arad, Sibiu, Oradea]
[Arad, Sibiu, Arad]
[Arad, Timisoara, Lugoj, Mehadia]
[Arad, Timisoara, Lugoj, Timisoara]
[Arad, Timisoara, Arad, Timisoara]
[Arad, Timisoara, Arad, Zerind]
[Arad, Timisoara, Arad, Sibiu]
[Arad, Zerind, Oradea, Zerind]
[Arad, Zerind, Oradea, Sibiu]
[Arad, Zerind, Arad, Timisoara]
[Arad, Zerind, Arad, Zerind]
[Arad, Zerind, Arad, Sibiu]
[Arad, Sibiu, Rimnicu, Craiova]
[Arad, Sibiu, Rimnicu, Pitesti]
[Arad, Sibiu, Rimnicu, Sibiu]
[Arad, Sibiu, Fagaras, Bucharest]


## Section 3: Graph Search

In [33]:
import utils

def graph_search(problem):
    
    # Initialise Queue (FIFO)
    frontier = utils.FIFOQueue()
    frontier.append(Node(problem.initial_state))
    explored = set()
    iterations = []
    
    # Algorithm starts
    while  frontier:
        path = frontier.pop()
        explored.add(path.state)
        iterations.append(path.path())
        
        if problem.goal_test(path.state):
            return True, iterations, path
        
        paths = path.expand(problem)
        for p in paths:
            if p.state not in explored and p not in frontier:
                frontier.append(p)
        
    return False, None, None


In [34]:

gp = GraphProblem(g, 'Arad', 'Neamt')
result, iterations, path = graph_search(gp)
print(result)
print(path.path())
p = Plot()
p.plot(g)

for i in iterations:
    print(i)

def animate(i):
    p.plot(g, iterations[i])
    sites = iterations[i]
    for site in sites:
        x = romania_map_locations[site.state][0]
        y = romania_map_locations[site.state][1]
        s = p.sp.scatter(x, y, color='red', marker='o', s=100)
    return s

anim = animation.FuncAnimation(p.fig, animate, frames=len(iterations), interval=3000, blit=False)
HTML(anim.to_html5_video())
    


True
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi, Neamt]
[Arad]
[Arad, Timisoara]
[Arad, Zerind]
[Arad, Sibiu]
[Arad, Timisoara, Lugoj]
[Arad, Zerind, Oradea]
[Arad, Sibiu, Rimnicu]
[Arad, Sibiu, Fagaras]
[Arad, Timisoara, Lugoj, Mehadia]
[Arad, Sibiu, Rimnicu, Craiova]
[Arad, Sibiu, Rimnicu, Pitesti]
[Arad, Sibiu, Fagaras, Bucharest]
[Arad, Timisoara, Lugoj, Mehadia, Drobeta]
[Arad, Sibiu, Fagaras, Bucharest, Giurgiu]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Hirsova]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Hirsova, Eforie]
[Arad, Sibiu, Fagaras, Bucharest, Urziceni, Vaslui, Iasi, Neamt]
